<a href="https://colab.research.google.com/github/kyochanpy/Google_Smartphone_Decimeter_Challenge/blob/main/PP/position_shift_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install optuna
!pip install pyproj

In [2]:
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import os
from pathlib import Path
import pyproj
from pyproj import Proj, transform
import optuna


In [7]:
path = Path("/content/drive/MyDrive/GSDC")
test_base = pd.read_csv(path / "baseline_locations_test.csv")
sub = pd.read_csv(path / "sample_submission.csv")

truths = (path / "train").rglob("ground_truth.csv")

df_list = []
cols = ["collectionName", "phoneName", "millisSinceGpsEpoch", "latDeg", "lngDeg"]
sub_cols = sub.columns

for t in tqdm(truths, total=73):
    df_phone = pd.read_csv(t, usecols=cols)
    df_list.append(df_phone)
df_truth = pd.concat(df_list, ignore_index=True)

train_base = pd.read_csv(path / "baseline_locations_train.csv", usecols=cols)
all_df = df_truth.merge(train_base, how="inner", on=cols[:3], suffixes=("_truth", '_train_base'))

In [9]:
train_p_s = pd.read_csv(path / "baseline_locations_train.csv")
train_b = train_p_s[sub_cols]
test_b = test_base[sub_cols]

msge = "millisSinceGpsEpoch"

datadir = Path('/content/drive/MyDrive/GSDC')
testdir = datadir / 'test'
traindir = datadir / 'train'

g_t = pd.DataFrame()
for d in os.listdir(traindir):
    for p in os.listdir(traindir/d):
        g_t = g_t.append(pd.read_csv(traindir/d/p/'ground_truth.csv'))

g_t["phone"] = g_t["collectionName"] + "_" + g_t["phoneName"]
g_t_sub_cols = g_t[sub_cols]

In [10]:
def calc_haversine(lat1, lon1, lat2, lon2):
    RADIUS = 6_367_000
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    d = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    dist = 2 * RADIUS * np.arcsin(d**0.5)
    return dist

In [14]:
def position_shift(input_df):
    def compute_dist(fname_1, fname_2=g_t):
        oof = fname_1
        g_t = fname_2
        df = oof.merge(g_t, on=["phone", "millisSinceGpsEpoch"])
        dist_oof = calc_haversine(df.latDeg_x, df.lngDeg_x, df.latDeg_y, df.lngDeg_y)
        scores = pd.DataFrame({"phone":df.phone, "dist":dist_oof})
        scores_grp = scores.groupby("phone")
        d_50 = scores_grp.quantile(.50).reset_index()
        d_50.columns = ["phone", "q_50"]
        d_95 = scores_grp.quantile(.95).reset_index()
        d_95.columns = ["phone", "q_95"]
        return (scores_grp.quantile(.50).mean() + scores_grp.quantile(.95).mean()) / 2, d_50.merge(d_95)


    def WGS84_to_ECEF(lat, lon, alt):
        rad_lat = lat * (np.pi / 180.0)
        rad_lon = lon * (np.pi / 180.0)
        a = 6378137.0
        finv = 298.257223563
        f = 1 / finv   
        e2 = 1 - (1 - f) * (1 - f)    
        N = a / np.sqrt(1 - e2 * np.sin(rad_lat) * np.sin(rad_lat))
        x = (N + alt) * np.cos(rad_lat) * np.cos(rad_lon)
        y = (N + alt) * np.cos(rad_lat) * np.sin(rad_lon)
        z = (N * (1 - e2) + alt)        * np.sin(rad_lat)
        return x, y, z    


    transformer = pyproj.Transformer.from_crs(
        {"proj":'geocent', "ellps":'WGS84', "datum":'WGS84'},
        {"proj":'latlong', "ellps":'WGS84', "datum":'WGS84'},)


    def ECEF_to_WGS84(x, y, z):
        lon, lat, alt = transformer.transform(x, y, z, radians=False)
        return lon, lat, alt


    def position_shift_(input_df, a):
        output_df = input_df.copy()
        output_df["heightAboveWgs84EllipsoidM"] = 63.5
        output_df["x"], output_df["y"], output_df["z"] = zip(*output_df.apply(lambda x: WGS84_to_ECEF(x.latDeg, x.lngDeg, 
                                                                                                    x.heightAboveWgs84EllipsoidM), axis=1))
        output_df.sort_values(["phone", msge], inplace=True)
        for fi in ["x", "y", "z"]:
            output_df[[fi + "_p"]] = output_df[fi].shift().where(output_df["phone"].eq(output_df["phone"].shift()))
            output_df[[fi + "_diff"]] = output_df[fi] - output_df[fi + "_p"]

        output_df[["dist"]] = np.sqrt(output_df["x_diff"]**2 + output_df["y_diff"]**2 + output_df["z_diff"]**2)
        for fi in ["x", "y", "z"]:
            output_df[[fi + "_new"]] = output_df[fi + "_p"] + output_df[fi + "_diff"] * (1 - a/output_df["dist"])
        lng, lat, alt = ECEF_to_WGS84(output_df["x_new"].values, output_df["y_new"].values, output_df["z_new"].values)

        lat[np.isnan(lat)] = output_df.loc[np.isnan(lat), "latDeg"]
        lng[np.isnan(lng)] = output_df.loc[np.isnan(lng), "lngDeg"]
        output_df["latDeg"] = lat
        output_df["lngDeg"] = lng

        output_df.sort_values(["phone", msge], inplace=True)

        return output_df[sub_cols]


    def objective(trial):
        a = trial.suggest_uniform("a", -1, 1)
        score, scores = compute_dist(position_shift_(train_b, a), g_t)
        return score


    study = optuna.create_study()
    study.optimize(objective, n_trials=30)

    output_df = position_shift_(test_base, a=study.best_params["a"])

    return output_df

In [15]:
position_shift(test_base)

[I 2021-06-28 11:01:11,987] A new study created in memory with name: no-name-adad84b9-cce8-46f1-b26f-dd0f57afd3c0
[I 2021-06-28 11:01:18,418] Trial 0 finished with value: 5.2995641969863225 and parameters: {'a': -0.03566780781469148}. Best is trial 0 with value: 5.2995641969863225.
[I 2021-06-28 11:01:24,723] Trial 1 finished with value: 5.703680954295509 and parameters: {'a': -0.9143048452973408}. Best is trial 0 with value: 5.2995641969863225.
[I 2021-06-28 11:01:31,065] Trial 2 finished with value: 5.196920250979717 and parameters: {'a': 0.8200065698472088}. Best is trial 2 with value: 5.196920250979717.
[I 2021-06-28 11:01:37,405] Trial 3 finished with value: 5.190831528079224 and parameters: {'a': 0.5715046609032228}. Best is trial 3 with value: 5.190831528079224.
[I 2021-06-28 11:01:43,580] Trial 4 finished with value: 5.628926388494557 and parameters: {'a': -0.7836575687733274}. Best is trial 3 with value: 5.190831528079224.
[I 2021-06-28 11:01:49,936] Trial 5 finished with valu

,phone,millisSinceGpsEpoch,latDeg,lngDeg
0,2020-05-15-US-MTV-1_Pixel4,1273608785432,37.416628,-122.082053
1,2020-05-15-US-MTV-1_Pixel4,1273608786432,37.416641,-122.082044
2,2020-05-15-US-MTV-1_Pixel4,1273608787432,37.416647,-122.082040
3,2020-05-15-US-MTV-1_Pixel4,1273608788432,37.416613,-122.082060
4,2020-05-15-US-MTV-1_Pixel4,1273608789432,37.416608,-122.082065
...,...,...,...,...
91481,2021-04-29-US-SJC-3_SamsungS20Ultra,1303763185000,37.334544,-121.899387
91482,2021-04-29-US-SJC-3_SamsungS20Ultra,1303763186000,37.334539,-121.899383
91483,2021-04-29-US-SJC-3_SamsungS20Ultra,1303763187000,37.334547,-121.899377
91484,2021-04-29-US-SJC-3_SamsungS20Ultra,1303763188000,37.334546,-121.899371


In [18]:
train_b

,collectionName,phoneName,millisSinceGpsEpoch,latDeg,lngDeg
0,2020-05-14-US-MTV-1,Pixel4,1273529463442,37.423575,-122.094091
1,2020-05-14-US-MTV-1,Pixel4,1273529464442,37.423578,-122.094101
2,2020-05-14-US-MTV-1,Pixel4,1273529465442,37.423573,-122.094111
3,2020-05-14-US-MTV-1,Pixel4,1273529466442,37.423583,-122.094121
4,2020-05-14-US-MTV-1,Pixel4,1273529467442,37.423579,-122.094114
...,...,...,...,...,...
131337,2021-04-29-US-SJC-2,SamsungS20Ultra,1303760315000,37.334460,-121.899600
131338,2021-04-29-US-SJC-2,SamsungS20Ultra,1303760316000,37.334472,-121.899583
131339,2021-04-29-US-SJC-2,SamsungS20Ultra,1303760317000,37.334491,-121.899597
131340,2021-04-29-US-SJC-2,SamsungS20Ultra,1303760318000,37.334495,-121.899583


In [17]:
test_b

,collectionName,phoneName,millisSinceGpsEpoch,latDeg,lngDeg
0,2020-05-15-US-MTV-1,Pixel4,1273608785432,37.416628,-122.082053
1,2020-05-15-US-MTV-1,Pixel4,1273608786432,37.416646,-122.082040
2,2020-05-15-US-MTV-1,Pixel4,1273608787432,37.416653,-122.082039
3,2020-05-15-US-MTV-1,Pixel4,1273608788432,37.416607,-122.082063
4,2020-05-15-US-MTV-1,Pixel4,1273608789432,37.416609,-122.082073
...,...,...,...,...,...
91481,2021-04-29-US-SJC-3,SamsungS20Ultra,1303763185000,37.334539,-121.899383
91482,2021-04-29-US-SJC-3,SamsungS20Ultra,1303763186000,37.334545,-121.899380
91483,2021-04-29-US-SJC-3,SamsungS20Ultra,1303763187000,37.334551,-121.899371
91484,2021-04-29-US-SJC-3,SamsungS20Ultra,1303763188000,37.334540,-121.899371
